# Задания

In [1]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg
import warnings
warnings.filterwarnings("ignore")
import pycountry
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.cluster import MeanShift

## Задание 1

Загрузи данные о посещениях заведений. В них содержится информация о регистрации пользователя в заведениях 
и геолокация этих заведений. Очисти данные от записей с пропусками. \
Выведи количество записей после очистки. 

In [2]:
df = pd.read_csv("../datasets/checkins.dat", sep="|", skipinitialspace=True)
df.dropna(inplace=True)
print("Количество записей после очистки:", len(df))

Количество записей после очистки: 396634


## Задание 2

Эти данные содержат информацию о заведениях со всего мира. С помощью геолокаций и библиотеки [Reverse Geocoder](https://github.com/thampiman/reverse-geocoder),
узнай страну для каждой геопозиции. \
Узнай **название** второй страны по количеству записей.

In [3]:
df.columns = df.columns.str.strip()

# извлечение геопозиций
locations = df[['latitude', 'longitude']].values.tolist()

# Изменение формата передачи координат
locations = [(location[0], location[1]) for location in locations]

# использование reverse_geocoder для определения страны для каждой геопозиции
results = rg.search(locations)

# создание словаря для хранения количества записей для каждой страны
countries = {}
for result in results:
    country_code = result['cc']
    if country_code in countries:
         countries[country_code] += 1
    else:
         countries[country_code] = 1

# сортировка по количеству записей и получение названия второй страны
top_countries = sorted(countries.items(), key=lambda x: x[1], reverse=True)
top_two_countries = top_countries[:2]
second_country_name = pycountry.countries.get(alpha_2=top_two_countries[1][0]).name

#Cчитаем записи для первых двух стран
for i in range(2):
    country_code = top_two_countries[i][0]
    country_name = pycountry.countries.get(alpha_2=country_code).name
    count = top_two_countries[i][1]
    print(f"{country_name}: {count}")
    
# вывод названия второй страны
second_country_name = pycountry.countries.get(alpha_2=top_two_countries[1][0]).name
print("Название второй страны: ", second_country_name)


Loading formatted geocoded file...
United States: 373258
Indonesia: 5356
Название второй страны:  Indonesia


In [4]:
print(countries)

{'US': 373258, 'CO': 68, 'CA': 5341, 'MX': 833, 'IE': 117, 'PR': 245, 'AR': 174, 'AU': 528, 'CH': 149, 'GB': 1812, 'JP': 383, 'KR': 267, 'RU': 157, 'BR': 720, 'NL': 338, 'NZ': 77, 'FR': 494, 'FI': 137, 'IT': 252, 'IL': 187, 'SE': 109, 'DE': 552, 'ID': 5356, 'UY': 3, 'TR': 100, 'PT': 35, 'PA': 35, 'IN': 212, 'ES': 316, 'AT': 63, 'SG': 522, 'NO': 27, 'KE': 374, 'TW': 55, 'DO': 52, 'CL': 137, 'HU': 59, 'AE': 88, 'PH': 225, 'GR': 80, 'SA': 13, 'UA': 21, 'GG': 3, 'CN': 298, 'IS': 16, 'ZA': 66, 'MY': 375, 'TT': 15, 'TH': 633, 'HR': 42, 'JE': 11, 'HK': 229, 'BH': 6, 'NI': 10, 'EG': 28, 'BE': 121, 'QA': 14, 'PL': 50, 'DK': 115, 'GT': 8, 'SK': 13, 'CZ': 30, 'VE': 22, 'CR': 38, 'PE': 48, 'EC': 19, 'EE': 14, 'SI': 9, 'SM': 1, 'BJ': 1, 'RO': 11, 'KW': 14, 'VI': 18, 'LB': 21, 'BS': 34, 'BB': 6, 'LV': 2, 'BZ': 8, 'VN': 28, 'CY': 8, 'IQ': 1, 'MC': 11, 'VA': 8, 'CU': 3, 'LU': 11, 'TL': 4, 'JM': 33, 'UZ': 7, 'IR': 7, 'MA': 5, 'MO': 2, 'AO': 1, 'AW': 7, 'NA': 5, 'PK': 5, 'HT': 5, 'BM': 11, 'JO': 10, 'AG

## Задание 3

Нас будут интересовать только американские локации. Очисти данные от локаций, находящихся в других странах. 
Также, чтобы сократить количество, геолокаций оставь в выборке только 50 самых часто встречаемых заведений (venue). \
Выведи количество локаций, оставшихся после этих очисток.

In [5]:
# загружаем данные и очищаем их
data = pd.read_csv("../datasets/checkins.dat", sep="|", skipinitialspace=True)
data.columns = data.columns.str.strip()
data.dropna(inplace=True)

# оставляем только локации, находящиеся в США
locations = [(lat, lon) for lat, lon in zip(data['latitude'], data['longitude'])]
results = rg.search(locations)
data_usa = data[[result['cc'] == 'US' for result in results]]

# выбираем только 50 самых часто встречаемых заведений
venues = data_usa['venue_id'].value_counts().head(50).index
data_usa_venues = data_usa[data_usa['venue_id'].isin(venues)]

# выводим количество локаций, оставшихся после очисток
print(f"Количество локаций: {len(data_usa_venues)}")

Количество локаций: 162099


## Задание 4

Перейдем к задаче кластеризации. Воспользуйся алгоритмом [Mean Shift](https://scikit-learn.org/stable/modules/clustering.html#mean-shift)
для кластеризации локаций. Параметрами укажи `MeanShift(bandwidth=0.1, bin_seeding=True)`. 

    `bandwidth=0.1` - это ширина ядра кластеризации. Для средних широт США - это порядка 5-10 км. 
    `bin_seeding=True` - для ускорения работы алгоритма.
    
Выведи количество кластеров, которые у тебя получились в результате кластеризации.

In [6]:
# кластеризация с помощью Mean Shift
bandwidth = 0.1
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True).fit(data_usa_venues[['latitude', 'longitude']])
labels = ms.labels_
n_clusters = len(np.unique(labels))

# вывод количества кластеров
print(f"Количество кластеров: {n_clusters}")

Количество кластеров: 2846


## Задание 5

Центры полученных кластеров - это потенциальные места установки банеров компании. Теперь хотелось бы найти те центры кластеров, 
которые наиболее близко находятся к офисам продаж компании. \
Загрузи [данные по координатам офисов компании](datasets/offices.csv). Для каждого офиса найди 5 самых ближайших к нему центров кластеров. 
(Пренебрежем тем, что Земля круглая и рассчитаем Евклидово расстояние).
У компании 11 офисов, значит у нас должно получится 55 мест установки баннеров. \
Выведи координаты установки баннера, который ближе всего находится к офису компании.

In [7]:
print(pd.read_csv("../datasets/offices.csv"))
offices = pd.read_csv ("../datasets/offices.csv")

    Unnamed: 0   latitude   longitude
0            0  37.805200 -122.401462
1            1  34.272249 -118.468420
2            2  33.751277 -118.188740
3            3  32.717588 -117.174758
4            4  29.307715  -94.797707
5            5  30.686856  -88.038649
6            6  27.945382  -82.443917
7            7  25.852788  -80.191339
8            8  30.406825  -81.578839
9            9  32.786669  -79.928605
10          10  39.264089  -76.589212


In [8]:
# загрузка данных об офисах компании
offices = pd.read_csv("../datasets/offices.csv")

# функция для подсчета расстояния между двумя точками
def distance(lat1, lon1, lat2, lon2):
    return ((lat1-lat2)**2 + (lon1-lon2)**2)**0.5

# инициализация списка для хранения ближайших центров кластеров
nearest_centers = []

# проходимся по каждому офису и находим 5 ближайших центров кластеров
for office in offices[['latitude', 'longitude']].values:
    distances = []
    for center in ms.cluster_centers_:
        dist = distance(office[0], office[1], center[0], center[1])
        distances.append(dist)
    nearest_centers_indices = np.argsort(distances)[:5]
    nearest_centers_coords = ms.cluster_centers_[nearest_centers_indices]
    nearest_centers.append(nearest_centers_coords)

In [9]:
# вывод координат ближайших к офису центров кластера
for i, nearest in enumerate(nearest_centers):
    office_coord = offices[['latitude', 'longitude']].values[i]
    distances = []
    for center in nearest:
        dist = distance(office_coord[0], office_coord[1], center[0], center[1])
        distances.append(dist)
    nearest_center_index = np.argmin(distances)
    nearest_center_coord = nearest[nearest_center_index]
    print(f"Кластер {i+1}: ({round(nearest_center_coord[0], 6)}, {round(nearest_center_coord[1], 6)})")

Кластер 1: (37.688043, -122.409142)
Кластер 2: (34.187689, -118.448805)
Кластер 3: (33.809806, -118.144971)
Кластер 4: (32.715963, -117.158197)
Кластер 5: (29.301348, -94.797696)
Кластер 6: (30.694357, -88.043054)
Кластер 7: (27.949461, -82.464971)
Кластер 8: (25.786986, -80.218559)
Кластер 9: (30.332432, -81.654927)
Кластер 10: (32.785318, -79.924742)
Кластер 11: (39.287294, -76.613493)


In [10]:
# находим ближайший баннер к офису
min_distance = float('inf')
nearest_banner_coord = None
for i, nearest in enumerate(nearest_centers):
    office_coord = offices[['latitude', 'longitude']].values[i]
    for center in nearest:
        dist = distance(office_coord[0], office_coord[1], center[0], center[1])
        if dist < min_distance:
            min_distance = dist
            nearest_banner_coord = center

# вывод координат ближайшего баннера
print(f"Ближайший баннер к офису компании: ({round(nearest_banner_coord[0], 6)}, {round(nearest_banner_coord[1], 6)})")

Ближайший баннер к офису компании: (32.785318, -79.924742)


## Задание 6

С помощью функции [scatter_mapbox](https://plotly.github.io/plotly.py-docs/generated/plotly.express.scatter_mapbox.html) 
отметь точки установки баннеров. У тебя должна получится такая картинка.
Цветом точки укажи к какому офису будет относиться этот баннер. 
>Цвет легенды может отличаться от референса

<center><img src="../misc/images/task_6.png" width="800" height="800"/> <center/>

In [11]:
import plotly.express as px

# загрузка данных об офисах компании
offices = pd.read_csv("../datasets/offices.csv")

# функция для подсчета расстояния между двумя точками
def distance(lat1, lon1, lat2, lon2):
    return ((lat1-lat2)**2 + (lon1-lon2)**2)**0.5

# инициализация списка для хранения ближайших центров кластеров
nearest_centers = []

# проходимся по каждому офису и находим 5 ближайших центров кластеров
for office in offices[['latitude', 'longitude']].values:
    distances = []
    for center in ms.cluster_centers_:
        dist = distance(office[0], office[1], center[0], center[1])
        distances.append(dist)
    nearest_centers_indices = np.argsort(distances)[:5]
    nearest_centers_coords = ms.cluster_centers_[nearest_centers_indices]
    nearest_centers.append(nearest_centers_coords)

# создание датафрейма с координатами баннеров и офисов
df = pd.DataFrame(columns=['latitude', 'longitude', 'index_office'])
for i, nearest in enumerate(nearest_centers):
    office = offices[['latitude', 'longitude']].values[i]
    for center in nearest:
        df = df.append({'latitude': center[0], 'longitude': center[1], 'index_office': f"{i+1}"}, ignore_index=True)

# создание интерактивной карты с точками баннеров
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", hover_name="index_office", color="index_office",
                        height=600, zoom=2)
fig.update_layout(mapbox_style="open-street-map")
fig.show()


AttributeError: 'DataFrame' object has no attribute 'append'